In [12]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/1', creation_time=1717343602995, experiment_id='1', last_update_time=1717343602995, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Load the data

In [2]:
import pandas as pd

In [3]:
def read_data(url: str) -> pd.DataFrame:
    df = pd.read_parquet(url)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda t: t.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
train_data = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
val_data = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

print(len(train_data), len(val_data))

2898906 2938060


In [6]:
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error

## Model experiment

In [7]:
df_train = train_data
df_val = val_data

In [10]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)
y_train = df_train[target].values

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)
y_val = df_val[target].values

In [13]:
with mlflow.start_run():
    mlflow.set_tag("developer", "nda")
    mlflow.log_param("train_data_path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("val_data_path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

    mlflow.log_param("mode", "linear_reg")
    model = LinearRegression()
    model = model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    print("mse: ", mse)
    print("rmse: ", rmse)

mse:  66.03789333423961
rmse:  8.126370243487532


In [ ]:
    with open('./models/lin_reg.bin', 'wb') as f_out:
        pickle.dump((dv, model), f_out)